In [4]:
import xarray as xr
import dask
import numpy as np
import os
import time
import glob
from datetime import date
today=date.today()

import sys
sys.path.insert(0,'/home/albert7a/git/xscale')
import xscale

from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=1,name='make_profiles',walltime='00:20:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1','--ntasks-per-node=24'],memory='120GB',interface='ib0') 
print(cluster.job_script()) 

cluster.scale(240) 



#!/usr/bin/env bash

#SBATCH -J make_profiles
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=112G
#SBATCH -t 00:20:00
#SBATCH --constraint=HSW24
#SBATCH --exclusive
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=24
JOB_ID=${SLURM_JOB_ID%;*}



/scratch/cnt0024/hmg2840/albert7a/anaconda3/bin/python -m distributed.cli.dask_worker tcp://172.30.100.3:41696 --nthreads 0 --nprocs 28 --memory-limit 4.29GB --name make_profiles--${JOB_ID}-- --death-timeout 60 --local-directory $SCRATCHDIR/daskjobqueue --interface ib0 --interface ib0



/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")


In [5]:
cluster

In [7]:
from dask.distributed import Client
client = Client(cluster)


In [8]:
client

Client Scheduler: tcp://172.30.100.3:41696 Dashboard: http://172.30.100.3:45290/status,Cluster Workers: 252 Cores: 252 Memory: 1.08 TB


In [9]:
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)


In [10]:
def time_filt_aco():

    fileUtide="/store/albert7a/eNATL60/eNATL60-BLBT02-S/1h/surf/eNATL60ACO-BLBT02_y2009m10d29.1h_sozocrtx.nc"
    fUtide = xr.open_mfdataset(fileUtide,chunks={'x':500,'y':500})
    Utide=fUtide['sozocrtx'][:]
    navlat= fUtide['nav_lat'][0]
    navlon= fUtide['nav_lon'][0]
    T=2*np.pi/(1E-4)

    wutide=Utide.window
    wutide.set(n=48,dim='time_counter', cutoff=2*T)
    Utidefilt=wutide.convolve()

    dirfilttide='/scratch/cnt0024/hmg2840/albert7a/eNATL60/scale/'

    in_Utide_filt=fileUtide
    dstin=Dataset(in_Utide_filt,'r')
    out_Utide_filt='eNATL60ACO-BLBT02_1h_20090710_20090809_gridU-2D_filt2T.nc'
    dstout=Dataset(dirfilttide+out_Utide_filt,'w')

    varname='sozocrtx'

    today=date.today()
    dstout.description = "Data time filtered with Lanczos filter with window size of "+str(30)+" and cut-off frequency of "+str(T)+" obtained "+str(today.day)+"/"+str(today.month)+"/"+str(today.year)

    for dname, the_dim in dstin.dimensions.iteritems():
        dstout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)

    for v_name, varin in dstin.variables.iteritems():
        if v_name == varname:
            continue
        outVar = dstout.createVariable(v_name, varin.datatype, varin.dimensions)
        outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
        outVar[:] = varin[:]

    datain=dstin[varname]
    dataout=dstout.createVariable(varname+'_filt',datain.datatype,datain.dimensions)
    dataout.setncatts({k: datain.getncattr(k) for k in datain.ncattrs()})

    ztemp=np.zeros(datain.shape)
    ztemp[:]=Utidefilt[:]
    
    dataout[:]=ztemp[:]
    dstout.close()

read the data
filter data
to dataset
to netcdf


distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

all done
already done
CPU times: user 11min, sys: 33.2 s, total: 11min 33s
Wall time: 14min 40s


In [ ]:
%time time_filt_aco()